In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import sqlite3
from bs4 import BeautifulSoup

from main import main, main_cluster
from src.utils.helpers import truncate_text, clean_string
from src.utils.logger import setup_logger
from src.models.llm.access_2_cluster import Access2Cluster

logger = setup_logger(__name__, level='DEBUG') # Change to 'INFO' for less verbosity

ModuleNotFoundError: No module named 'playwright'

# Load Data

In [4]:
# Connect to the database
conn = sqlite3.connect('../data/raw/playwright_script.db')
cursor = conn.cursor()

res = cursor.execute("SELECT * FROM tests")
items = res.fetchall()

print("There are {} data.".format(len(items)))

There are 123 data.


In [5]:
# Check the first item
items[0]

('1.1',
 '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte".',
 '[1.1] Expected result: Die Arbeitsmappe wird geöffnet, der Analysekontext ist nicht sichtbar.',
 '.\\html\\1_1.html',
 '.\\screenshot\\1_1.png',
 '.\\test_script\\1_1.spec.ts')

# Approach

Use a Pre-trained LLM:
* GPT-3, GPT-3.5, or a lighter version like GPT-2 (well-suited for text generation tasks)
* Fine-tuning or adapting for specific tasks later possible

Steps:
1. ✅ HTML Processing: Extract relevant information from the HTML file.
    * Use **BeautifulSoup** or lxml in Python to parse and extract information from the HTML file. ➡️ see src.data.html_processor.py
2. ✅ Image Processing: Extract relevant information from the image:
    * Use image-to-text models like **Tesseracts** or pytesseract (OCR libraries) to extract text from the image. ➡️ see src.data.image_processor.py
    * Use OpenCV or PIL (Pillow) in Python to process the image and extract relevant information.
3. Summarize the image and HTML information and the prompt from the playwright test code using T5 model. (optional)
4. ✅ Python Processing: Parse the given playwright test code for previous step as a precondition. ➡️ see src.data.python_processor.py
5. ✅ Combine the extracted information from the HTML and the image with the prompt for the language model. ➡️ see src.data.input_combiner.py
6. ✅ Pass the combined input to the language model for generating the UI test code. ➡️ see src.ui_tests.test_generation.py

➡️ Run locally via notebook or script using the src.main.py

# HTML Processing

In [9]:
# load example file
html_path = '../data/raw/html/0_1.html'

Option 1: Parse all HTML content by extracting the text from it using BeautifulSoup:

In [10]:
from src.data.html_processor import parse_html
html_text = parse_html(html_path, max_length=2000)
print(html_text)

Startseite - disy CadenzaAchtung!Ihr Browser unterstÃ¼tzt kein JavaScript oder JavaScript wurde in Ihrem Browser deaktiviert.Bitte verwenden Sie einen Browser, der JavaScript unterstÃ¼tzt, oder aktivieren Sie JavaScript in Ihrem Browser. Ohne aktiviertes JavaScript ist die Anwendung nicht nutzbar.OfflineVerbinden â€¦Zum Navigatorbaum springenZum Hauptbereich springenStartseiteKartedisy Cadenza[{"printName":"Lernmodule â€“ Tutorials und mehr","url":"/help-learning/index.html","targetFrame":"_blank","id":"help","type":"help","webApplication":false},{"printName":"Hilfe","url":"/help/","targetFrame":"_blank","id":"help","type":"help","webApplication":false},{"printName":"Hilfe zu Classic","url":"/help-classic/","targetFrame":"_blank","id":"help-classic","type":"help-classic","webApplication":false}]Admin[{"printName":"Profil","url":"/pages/access/userprofile.xhtml","targetFrame":"_self","id":"userprofile","type":"userprofile","webApplication":false},{"printName":"Abmelden","url":"/logout",

Option 2: Extract only Elements (input fields, buttons, links) from the HTML file using BeautifulSoup.

In [13]:
from src.data.html_processor import extract_html_info
html_elements = extract_html_info(html_path, concat_mod='all')
print(html_elements)

Buttons: 
Button ids: navigationTrigger, workbook-create, RDxYr2vFytOijWjelj7P1
Button classes: button button-icon button-borderless, button workbook-create button-icon, button navigation-menu button-icon, button, button
Button texts: Arbeitsmappe importieren, Repository neu einlesen …
Inputs: 
Input classes: select2-search__field
Input aria-labels: Suchen nach …
Input types: search
Input placeholders: Suchen nach …
Links: 
Link texts: Zum Navigatorbaum springen, Zum Hauptbereich springen, Startseite, Karte, Verzeichnis Tutorial, Verzeichnis Gewässergüte, Verzeichnis Automobile, Verzeichnis Ergänzende Geodaten, Verzeichnis Zentrale Dienste, Verzeichnis Meine Arbeitsmappen, Arbeitsmappe Zugangsdaten, Zugangsdaten, disy Cadenza, Tutorials, Lernmodulen, Onlinehilfe, Webseite, Lernmodulen, 1, 2, 3, 4, 5, 6, 7, disy Cadenza v9.4.71, © Disy Informationssysteme GmbH, Über Disy
Link ids: skip-to-navigator, skip-to-content, home
Link classes: button button-primary, button button-primary, button

# (Local) Pipeline: LLM UI Test Generation
Using smaller GPT-2 model

1. First, extract the relevant information from the database for a specific test to be predicted:

In [10]:
def get_previous_id(id):
    test, step = map(int, id.split('.'))
    if step > 1:
        previous_id = f"{test}.{step - 1}"
    else:
        print(f"Test {test} has no more previous step. No context available.")

    return previous_id

In [11]:
def fetch_relevant_items(db_file, current_id):
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the previous ID
    previous_id = get_previous_id(current_id)

    # Prepare the SQL query to retrieve the desired rows
    query = 'SELECT * FROM tests WHERE id IN (?, ?)'
    cursor.execute(query, (current_id, previous_id))

    # Fetch all matching rows
    items = cursor.fetchall()

    # Close the connection
    conn.close()

    return items

In [12]:
def map_items_to_args(items):
    # The file paths from the test x-1 are used as context
    html_path = items[0][3]
    image_path = items[0][4]
    precondition_path = items[0][5]
    # The last step of the text x is used as prompt
    steps = items[1][1].split(']')
    description = steps[-1].strip()

    return {"html_path": html_path,
            "image_path": image_path,
            "precondition_path": precondition_path,
            "description": description}

In [13]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [14]:
# Get relevant data from the database
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [15]:
items

[('1.3',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen"',
  '[1.3] Expected result: Kontextmenü erscheint mit den Optionen: - Duplizieren - In Arbeitsblatt duplizieren - Exportieren - Designer öffnen - Löschen.',
  '.\\html\\1_3.html',
  '.\\screenshot\\1_3.png',
  '.\\test_script\\1_3.spec.ts'),
 ('1.4',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen". [1.4] Klicke auf "Duplizieren".',
  '[1.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in

In [16]:
args

{'html_path': '.\\html\\1_3.html',
 'image_path': '.\\screenshot\\1_3.png',
 'precondition_path': '.\\test_script\\1_3.spec.ts',
 'description': 'Klicke auf "Duplizieren".'}

2. Now we can run the main function to sum it up with the extracted information, we will get the generated UI test code for the test with id 1.4. Prompt engineering is implemented in the main function.

In [36]:
%%time
# Generate UI test code
prompt = main(**args)

2024-06-25 13:32:45 [src.main:25] [INFO] >>>> Loading context...
2024-06-25 13:32:45 [src.data.html_processor:106] [DEBUG] >>>> HTML elements extracted successfully. - Number of Elements: 36 - Number of Characters: 3522
2024-06-25 13:32:46 [src.data.image_processor:27] [DEBUG] >>>> Text extracted from image successfully. - Characters: 303
2024-06-25 13:32:46 [src.data.python_processor:15] [DEBUG] >>>> Python code parsed successfully. - Lines of Code: 17
2024-06-25 13:32:46 [src.main:31] [INFO] >>>> Context loaded successfully.
2024-06-25 13:32:46 [src.main:34] [INFO] >>>> Creating input prompt...
2024-06-25 13:32:46 [src.main:36] [INFO] >>>> Input prompt created successfully.
2024-06-25 13:32:46 [src.main:37] [DEBUG] >>>> Input prompt:
You are a test automation script writer. Your task is to create a Playwright test script for the given webpage. Below are the details of the webpage. Use this information to generate a complete Python Playwright test script following the instructions.

#

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 1024, but `max_length` is set to 500. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

# (Cluster) Pipeline: LLM UI Test Generation
Using Llava 1.5 model

1. Extract relevant information from the database for a specific test to be predicted:

In [166]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [167]:
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [168]:
items

[('1.3',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen"',
  '[1.3] Expected result: Kontextmenü erscheint mit den Optionen: - Duplizieren - In Arbeitsblatt duplizieren - Exportieren - Designer öffnen - Löschen.',
  '.\\html\\1_3.html',
  '.\\screenshot\\1_3.png',
  '.\\test_script\\1_3.spec.ts'),
 ('1.4',
  '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte". [1.2]  Öffnen der Tabellen-Sicht "Messstellenliste" über die Werkzeugliste der Arbeitsmappe. [1.3] Klicke auf den 3-Punkte-Button innerhalb der Tabellen-Sicht "Messstelleninformationen". [1.4] Klicke auf "Duplizieren".',
  '[1.4] Expected result: Das Kontextmenü schließt sich. Die Sicht ist dupliziert. Die duplizierte Sicht erscheint neben der Original-Sicht. Beide Sichten teilen sich in

In [169]:
# To traverse folders in cluster change path
args['image_path'] = args['image_path'].replace('\\', '/')

In [170]:
args

{'html_path': '.\\html\\1_3.html',
 'image_path': './screenshot/1_3.png',
 'precondition_path': '.\\test_script\\1_3.spec.ts',
 'description': 'Klicke auf "Duplizieren".'}

2. Setup access to the cluster:

Precondition:
- Adjust the paths in src file with your specific student credentials
- Make sure that playwright is installed on your local machine

In [171]:
access2cluster = Access2Cluster()
await access2cluster.login()

NotImplementedError: 

3. Init the Llava 1.5 model:

In [ ]:
await access2cluster.start_llm()
args['model'] = access2cluster

In [ ]:
args

4. Run the main function to sum it up with the extracted information, we will get the generated UI test code for the test with id 1.4. Prompt engineering is implemented in the main function.

In [ ]:
await main_cluster(**args)

# More Developing: Using modular source code
Possible to update function content in the following files, but no RENAMING or DELETING or changing function signature!:
* src/main.py: Main function for UI test generation.
    * Change constant Max length (max length of the input text for the model)
* src/ui_tests/test_generation.py: Generate the UI test code using the language model.
    * Expand model selection: Just add new case statements for different models with its code.
* src/data/html_processing.py: Extract relevant information from the HTML file.
* src/data/image_processing.py: Extract relevant information from the image.
* src/data/python_processing.py: Parse the given playwright test code for previous step as a precondition.
